In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt
from tqdm import tqdm
import random

random_state=0
np.random.seed(random_state)
random.seed(random_state)

from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import silhouette_score
from sklearn.cluster import KMeans
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler
import scipy.cluster.hierarchy as shc
import scipy.cluster.hierarchy as hcluster
from sklearn.cluster import AgglomerativeClustering
from scipy.cluster.hierarchy import dendrogram
from kmodes.kprototypes import KPrototypes

## B2B Customer Clustering for new TMS service Planning 

#### Service Description

***'TMS'*** is air conditioner(A/C) maintenance service of LG which targets business customers in forms of subscription.
    
In TMS service, there are currently 7 types of service which customers can subscribe, G1, G2, G3, T1, T2, T3, and R(***See attached file 'TMS_service_description.pdf'***). Each service differentiates in terms of the subscription benefits containing the cost of repairs(ex._free repair for specific failure symptoms), provision of additional service options like remote control, energy management, and periodic inspections.    
     
TMS has the weakness that the existing service could not reflect the various characteristics(ex A/C user pattern) of real customer segments or personalized characteristics of a specific segment. The need for planning specialized tms service for real customers is increasing, based on analysis of real b2b data.    
  
We will newly define customer segments and analyze each segment's unmet needs by clustering LG B2B data. THIS CAN BE A NEW OPPORTUNITY of NEW TMS SERVICE PLANNING OR NEW CUSTOMER SERVICE.

Dataset we will use includes
- 1) each customer's TMS subscription information ex)TMS service currently subscribed to
- 2) each customer's air conditioner A/S record ex)Failure symptoms, which repair taken
- 3) each customer's individual information ex) business type, region

#### Feature Description

1) ***Building_use_type*** : b2b Customer's building/facility usage type like hospital, university, office. (=사용장소)   
2) ***total_idu*** : The number of indoor AC(air conditioners) units installed per each b2b customers.    
3) ***total_odu*** : The number of outdoor AC(air conditioners) units installed per each b2b customers.    
4) ***contract_tms*** : TMS service which each custromer are subscribing now.    
5) ***Region*** : The region b2b customer's building/facility located   (=지역)  
6) ***Failure_symptoms*** : A/C Failure Symptoms recorded by an A/S engineer   (=고장증상)  
7) ***Repair_code*** : General type of Repair taken by a engineer to resolve symptoms of failure.  (=처리코드)    
8) ***Repair_type*** : Specific type of Repair taken by a engineer. It is more specific than 'Repair_code'.    (=처리유형명)   
9) ***Defective_Type*** : Type of the Defection (the cause of A/C failure or the cause that customer call A/S)  (=불량구분명)   
10) ***Repair_duration*** : The total time taken from registration of A/S to completion of the A/S.    (=처리소요시간)   
11) ***Receipt_amount*** : The actual amount of fee paid by the customer, excluding the free benefits provided by 'TMS' service.  (=실영수금액)   


* ***id_site: It is a kind of UNIQUE CUSTOMER NUMBER. The Same ID_SITE means the SAME CUSTOMER. When you analyze the data, you can reflect the information freely or you can delete columns and regard each low as a cusomer as TA did in practice class*** )

### Data Loading

In [2]:
df = pd.read_excel('./(eng)week5.TMS_Customer_Data.xlsx', engine='openpyxl')

In [3]:
df.head()

,id_site,Building_use_type,total_idu,total_odu,contract_tms,Region,Failure_symptoms,Repair_code,Repair_type,Defective_Type,Repair_duration,Receipt_amount
0,10000391,Office,33.0,10.0,T3,Seoul,Heater error,component related repair,General_repair,Product defect,73.0,0
1,10000391,Office,33.0,10.0,T3,Seoul,Accessories/AC component,component related repair,General_repair,Product defect,5.0,0
2,10000391,Office,33.0,10.0,T3,Seoul,Cooler Error,explanation_processing,General_repair,Emotional dissatisfaction,1017.0,0
3,10000391,Office,33.0,10.0,T3,Seoul,Inconvenient use,component related repair,General_repair,Product defect,273.0,0
4,10000391,Office,33.0,10.0,T3,Seoul,Display window error,component related repair,General_repair,Product defect,343.0,0


In [4]:
#start your own code~